In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/20 19:58:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")


In [10]:
from pyspark.sql.functions import split, explode, col, desc

df_benign = df.where(col("Label") == "Benign")
df_ddos = df.where(col("Label") != "Benign")


In [11]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.functions import col, avg
from pyspark.sql.types import StringType, IntegerType, FloatType

#Comparações entre trafegos legitimos e ddos
#Media de Pkt len min (menor tamanho de um pacote em um fluxo) 

df_benign = df_benign.withColumn("Pkt Len Min",col("Pkt Len Min").cast(FloatType()))
df_ddos = df_ddos.withColumn("Pkt Len Min",col("Pkt Len Min").cast(FloatType()))

df_b = df_benign.select(avg(col("Pkt Len Min")))
df_d = df_ddos.select(avg(col("Pkt Len Min")))
df_b.show()
df_d.show()

+------------------+
|  avg(Pkt Len Min)|
+------------------+
|14.473342528764723|
+------------------+



+-------------------+
|   avg(Pkt Len Min)|
+-------------------+
|0.02044001472658713|
+-------------------+



+------------------+
|  avg(Pkt Len Max)|
+------------------+
|396.01948709106955|
+------------------+



+-----------------+
| avg(Pkt Len Max)|
+-----------------+
|468.0883803797735|
+-----------------+



In [ ]:
#Media de Pkt len max (maior tamanho de um pacote em um fluxo)

df_benign = df_benign.withColumn("Pkt Len Max",col("Pkt Len Max").cast(FloatType()))
df_ddos = df_ddos.withColumn("Pkt Len Max",col("Pkt Len Max").cast(FloatType()))

df_b = df_benign.select(avg(col("Pkt Len Max")))
df_d = df_ddos.select(avg(col("Pkt Len Max")))
df_b.show()
df_d.show()

In [12]:
#Media de Total pkts in forward direction

df_benign = df_benign.withColumn("TotLen Fwd Pkts",col("TotLen Fwd Pkts").cast(FloatType()))
df_ddos = df_ddos.withColumn("TotLen Fwd Pkts",col("TotLen Fwd Pkts").cast(FloatType()))

df_b = df_benign.select(avg(col("TotLen Fwd Pkts")))
df_d = df_ddos.select(avg(col("TotLen Fwd Pkts")))
df_b.show()
df_d.show()

+--------------------+
|avg(TotLen Fwd Pkts)|
+--------------------+
|  387.44362019493894|
+--------------------+



+--------------------+
|avg(TotLen Fwd Pkts)|
+--------------------+
|  1846.8235483875453|
+--------------------+

